In [1]:
import pandas as pd
import requests
import urllib # dataya erişim
import json # data formatının dönüşümü
import os
import glob



In [7]:

url = "https://api.ibb.gov.tr/havakalitesi/OpenDataPortalHandler/GetAQIStations"
def ibbVeriToDataframe(urladresi):
    # API bağlantısı
    sorgu = urllib.request.urlopen(urladresi)
    # Veriyi python-json dosya biçimine dönüştürme
    data = json.loads(sorgu.read().decode())
    # Sadece datanın olduğu bölümün alınması
    #data = data.get("value")
    # Verinin dataframe formatına dönüştürülmesi
    return pd.DataFrame(data)


In [8]:
df = ibbVeriToDataframe(url)
df.columns

Id_semt = df.set_index('Id')['Name'].to_dict()

In [9]:
for file_name in Id_semt.values():
    with open(f"{file_name}.csv", 'w') as f:
        pass  # Boş dosya oluşturur


In [10]:
base_url = "https://api.ibb.gov.tr/havakalitesi/OpenDataPortalHandler/GetAQIByStationId?StationId={}&StartDate={}%2000:00:00&EndDate={}%2000:00:00"
start_date = pd.to_datetime("2010-01-01")
end_date = pd.to_datetime("2023-01-01")
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

Burada veri çekerken bir takım sorunlar yaşadım, özellikle çok fazla istek yolladığımda karşılaştığım problemi gözlemleyebilmek için ve tekrar başlamamak için her bir ay için ayrı dosya çekmeyi uygun gördüm. 

In [14]:
for id, name in Id_semt.items():
    for i in range(len(date_range) - 1):
        start = date_range[i].strftime("%d.%m.%Y")
        end = date_range[i + 1].strftime("%d.%m.%Y")
        url = base_url.format(id, start, end)
        response = requests.get(url)
        # Yapılacak işlemler
        #df = ibbVeriToDataframe(url)

        # Her iterasyonda bir sonraki adımda kaydedilecek CSV dosyasının adı oluşturulur
        file_name = f"{name}{start}.csv"
        # Fonksiyon çalıştırılır ve df değişkenine atanır
        df = ibbVeriToDataframe(url)
        # CSV dosyası oluşturulur ve kaydedilir
        df.to_csv(f"dataset/{name}/{file_name}", index=False)
        #bir check kontrol yapılmalı..


nihai sonuçta bu csv dosyalarını birleştirmek için de bir kod blogu yazdım.

In [ ]:
path='/Users/*****/Desktop/Github/tubitak/dataset/'
# tüm klasörleri döngüye sok
for foldername in os.listdir(path):
    # her bir klasör için boş bir DataFrame oluştur
    combined_csv = pd.DataFrame()
    # klasördeki tüm CSV dosyalarını al
    csv_files = glob.glob(path + foldername + '/*.csv')
    # tarihe göre sırala
    csv_files.sort(key=os.path.getmtime)
    # her bir CSV dosyasını oku ve birleştir
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        combined_csv = pd.concat([combined_csv, df])
    # birleştirilmiş CSV dosyasını kaydet
    combined_csv.to_csv(path + foldername + '.csv', index=False)
